In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import math
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import sklearn.linear_model
from matplotlib.pylab import rcParams
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
%matplotlib inline
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris, make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor
from matplotlib.pylab import rcParams
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.io import loadmat
from sklearn.mixture import GaussianMixture
import imageio
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from PIL.ImageChops import add, subtract, multiply, difference, screen
import PIL.ImageStat as stat
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
from skimage import color, viewer, exposure, img_as_float, data
from skimage.transform import SimilarityTransform, warp, swirl
from skimage.util import invert, random_noise, montage
import matplotlib.image as mpimg
from scipy.ndimage import affine_transform, zoom
from scipy import misc
import matplotlib.cm as cm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from enum import Enum
from itertools import cycle, islice, dropwhile
import re

In [3]:
class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"

# 2-OPT Code

Source: https://stackoverflow.com/questions/53275314/2-opt-algorithm-to-solve-the-travelling-salesman-problem-in-python

Function Definition:

In [21]:
def cost(cost_mat, route):
    return cost_mat[np.roll(route, 1), route].sum()

def two_opt(cost_mat,route):
    best = route.copy()
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route)-2):
            for j in range(i+1, len(route)+1):
                if j-i == 1: continue # changes nothing, skip then
                new_route = route.copy()
                new_route[i:j] = route[j-1:i-1:-1] # this is the 2woptSwap
                if cost(cost_mat, new_route) < cost(cost_mat, best):
                    best = new_route
                    improved = True
        route = best
    return best, cost(cost_mat, best)

Graph Definition and Print Out

In [37]:
#Use this to read in data sets, replace "TSP1.txt" with the name of the file
data=np.array(pd.read_csv('3OPTBAD.txt', header=None, sep='\s+', engine='python'))


#If you want to manually build a distance matrix, you can do so here:
# data_mat = np.array([
# [0,2,80,1,80,80],
# [2,0, 2, 80,1,80],
# [80,2,0,2,80,1],
# [1,80,2,0,2,80],
# [80,1,80,2,0,2],
#     [2,80,1,80,2,0]
# ])


route=[]
route=[i for i in range(len(data))]

#change input to (data_mat,route) when using a manually built matrix 
Tour, Cost = two_opt(data,route)
print("Final Tour", Tour)
print("Final Cost", Cost)

Final Tour [0, 1, 2, 3, 4, 5, 6]
Final Cost 50


# 3-OPT

Source for the 3-OPT code: https://github.com/BraveDistribution/pytsp

Function Definition:

In [34]:
def cost(cost_mat, route):
    return cost_mat[np.roll(route, 1), route].sum()

def tsp_3_opt(graph, route=None):
    lengty=len(graph)
    route=[]
    route=[i for i in range(lengty)]
    moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                  OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                  OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0}
    improved = True
    best_found_route = route
    while improved:
        improved = False
        for (i, j, k) in possible_segments(len(graph)):
            # we check all the possible moves and save the result into the dict
            for opt_case in OptCase:
                moves_cost[opt_case] = get_solution_cost_change(graph, best_found_route, opt_case, i, j, k)
            # we need the minimum value of substraction of old route - new route
            best_return = max(moves_cost, key=moves_cost.get)
            if moves_cost[best_return] > 0:
                best_found_route = reverse_segments(best_found_route, best_return, i, j, k)
                improved = True
                break
    # just to start with the same node -> we will need to cycle the results.
    cycled = cycle(best_found_route)
    skipped = dropwhile(lambda x: x != 0, cycled)
    sliced = islice(skipped, None, len(best_found_route))
    best_found_route = list(sliced)
    return best_found_route


def possible_segments(N):
    """ Generate the combination of segments """
    segments = ((i, j, k) for i in range(N) for j in range(i + 2, N-1) for k in range(j + 2, N - 1 + (i > 0)))
    return segments


def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        return graph[A, B] + graph[E, F] - (graph[B, F] + graph[A, E])
    elif case == OptCase.opt_case_3:
        # ABC'
        return graph[C, D] + graph[E, F] - (graph[D, F] + graph[C, E])
    elif case == OptCase.opt_case_4:
        # A'BC'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[B, F] + graph[E, C])
    elif case == OptCase.opt_case_5:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[C, F] + graph[B, D] + graph[E, A])
    elif case == OptCase.opt_case_6:
        # AB'C
        return graph[B, A] + graph[D, C] - (graph[C, A] + graph[B, D])
    elif case == OptCase.opt_case_7:
        # AB'C'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[B, E] + graph[D, F] + graph[C, A])
    elif case == OptCase.opt_case_8:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[C, F] + graph[B, E])

def reverse_segments(route, case, i, j, k):
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
    return solution

Graph Initialization and Print Out

In [35]:
#Use this to read in data sets, replace "TSP1.txt" with the name of the file
data=np.array(pd.read_csv('3OPTBAD.txt', header=None, sep='\s+', engine='python'))

#If you want to manually build a distance matrix, you can do so here:
# graph = np.array([
# [0,2,80,1,80,80],
# [2,0, 2, 80,1,80],
# [80,2,0,2,80,1],
# [1,80,2,0,2,80],
# [80,1,80,2,0,2],
#     [2,80,1,80,2,0]
# ])

#change input to from "data" to "graph" when using a manually built matrix 
Tour = tsp_3_opt(data)
print("Final Tour", Tour)
print("Cost of Tour", cost(data, Tour))

Final Tour [0, 6, 1, 4, 5, 2, 3]
Cost of Tour 29


# TSP Code:

In [9]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

In [10]:
pip install python-tsp

Note: you may need to restart the kernel to use updated packages.


In [36]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming


# distance_matrix = np.array([
#     [0,  5, 4, 10],
#     [5,  0, 8,  5],
#     [4,  8, 0,  3],
#     [10, 5, 3,  0]
# ])
permutation, distance = solve_tsp_dynamic_programming(data)

print("Final Tour", permutation)
print("Total Cost", distance)

Final Tour [0, 1, 4, 5, 2, 6, 3]
Total Cost 28
